In [ ]:
from google.colab import drive
from bs4 import BeautifulSoup
import pandas as pd
import nltk                                # Python library for NLP
import re
import matplotlib.pyplot as plt
import time

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p /content/drive/MyDrive/Kaggle/stackoverflow/

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle/stackoverflow/

/content/drive/MyDrive/Kaggle/stackoverflow


In [ ]:
#download dataset
!kaggle datasets download -d stackoverflow/pythonquestions

 98% 547M/558M [00:05<00:00, 107MB/s]
100% 558M/558M [00:05<00:00, 113MB/s]


In [ ]:
# extract dataset and remove .zip files
!unzip \*.zip  && rm *.zip

Archive:  pythonquestions.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
replace Tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Tags.csv                


y

In [ ]:
%%time
ques = pd.read_csv("Questions.csv",engine='python')
ans = pd.read_csv("Answers.csv",engine='python')
tags = pd.read_csv("Tags.csv",engine='python')


CPU times: user 46.6 s, sys: 8.16 s, total: 54.8 s
Wall time: 54.6 s


In [ ]:
!mkdir -p /content/drive/MyDrive/Kaggle/stackoverflow/final_hard
%cd /content/drive/MyDrive/Kaggle/stackoverflow/final_hard

/content/drive/MyDrive/Kaggle/stackoverflow/final_hard


In [ ]:
total_ques_len = len(ques)
total_ans_len = len(ans)

print(f"Length of Questions in Total {total_ques_len}")
print(f"Length of Ans in Total {total_ans_len}")

Length of Questions in Total 607282
Length of Ans in Total 987122


In [ ]:
# Remove all questions that have a negative score
ques = ques[ques["Score"] >= 0]
# Remove answers of the removed question
ans = ans.loc[ans.ParentId.isin(ques.Id)]
len_ques_after_neg=len(ques)
len_ans_after_neg=len(ans)


In [ ]:
print(f"Length of Questions after removing negative score questions {len_ques_after_neg}")
print(f"Length of Answers after removing negative score questions {len_ans_after_neg}")


Length of Questions after removing negative score questions 570972
Length of Answers after removing negative score questions 927230


In [ ]:
# Remove Questions without answers
ques = ques.loc[ques.Id.isin(ans.ParentId)]
len_of_ques_after_no_answer_removal = len(ques)

print(f"Length of Questions after removing question with no answer {len_of_ques_after_no_answer_removal}")
print(f"Length of Questions after removing question with no answer {len(ans)}")


Length of Questions after removing question with no answer 503729
Length of Questions after removing question with no answer 927230


In [ ]:
per =(total_ques_len-len(ques))/len(ques) *100
print("persantage of questions removed: {0:.2g}%".format(per))
per =(total_ans_len-len(ans))/len(ans) *100
print("persantage of answers removed: {0:.2g}%".format(per))

persantage of questions removed: 21%
persantage of answers removed: 6.5%


#create dataset

In [ ]:
def get_text_from_html(html_string):
    """
    input:
        html_string (HTML string representing question body or answer body)
    return:
        html content
    """
    bs = BeautifulSoup(html_string,"html.parser")
    return bs.get_text()

In [ ]:
%%time
ques["BodyClean"]= ques['Body'].apply(get_text_from_html)

CPU times: user 3min 47s, sys: 1.27 s, total: 3min 48s
Wall time: 3min 48s


In [ ]:
ans

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,497,50.0,2008-08-02T16:56:53Z,469,4,<p>open up a terminal (Applications-&gt;Utilit...
1,518,153.0,2008-08-02T17:42:28Z,469,2,<p>I haven't been able to find anything that d...
2,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...
3,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...
4,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B..."
...,...,...,...,...,...,...
987117,40143290,3831.0,2016-10-19T23:46:58Z,40142906,0,<p>I am fairly certain your problem is your us...
987118,40143315,3125566.0,2016-10-19T23:49:43Z,40143166,2,"<p>First thing, you should use <code>if/elif</..."
987119,40143317,2350575.0,2016-10-19T23:50:04Z,40142194,0,<p>If you are using firefox ver >47.0.1 you ne...
987120,40143349,6934347.0,2016-10-19T23:54:02Z,40077010,0,<p>I solved my own problem defining the follow...


In [ ]:
#get top 1 answers of every question
top_ans = ans.sort_values(by="Score",ascending=False).groupby("ParentId").head(3).reset_index(drop=True)

In [ ]:
#get needed columns only
top_ans = top_ans[["Id", "Body","ParentId"]]
new_ques = ques[["Id","Title","BodyClean"]]
top_ans = top_ans.rename(columns={"Id":"ans_id"})

In [ ]:
ques.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,BodyClean
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,I am using the Photoshop's javascript API to f...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,I have a cross-platform (Python) application w...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,I'm starting work on a hobby project with a py...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,There are several ways to iterate over a resul...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,I don't remember whether I was dreaming or not...


In [ ]:
#positive examples
left = ques.set_index(['Id'])
right = top_ans.set_index(['ParentId'])

positive = left.join(right, lsuffix='Ques', rsuffix='Ans').reset_index()
positive = positive.rename(columns={"index":"ques_id","Title":"ques_title","BodyClean":"ques_body","BodyAns":"ans"})
positive = positive[["ques_id","ans_id","ques_title","ques_body","ans"]]
positive["is_answer"] = 1

In [ ]:
#negative examples
negative = positive[["ques_id","ques_title","ques_body"]]
shuffled_ans = ans.sample(frac=1).reset_index(drop=True)
shuffled_ans = shuffled_ans[:len(negative)]
negative = pd.concat([negative, shuffled_ans],axis=1)
negative=negative[["ques_id","Id","ques_title","ques_body","Body"]]
negative = negative.rename(columns={"Id":"ans_id","Body":"ans"})
negative["is_answer"]=0

In [ ]:
#merge negative and positive examples and suffle them
dataset = pd.concat([positive,negative])
dataset = dataset.sample(frac=1).reset_index(drop=True)

In [ ]:
#safe clean answer without html along with answer with html(if we wanted to remove codes)
dataset["ans"]= dataset['ans'].apply(get_text_from_html)

In [ ]:
%cd /content/drive/MyDrive/Kaggle/stackoverflow/

/content/drive/MyDrive/Kaggle/stackoverflow


In [ ]:
#save
dataset.to_pickle("./big_dataset.pkl")

In [ ]:
dataset

,ques_id,ans_id,ques_title,ques_body,ans,is_answer
0,12605170,17367020,Extract text per page with Python pdfMiner?,I have experimented with both pyPdf and pdfMin...,subprocess.call has the cwd keyword argument f...,0
1,16711325,25548040,python: iterate dictionary sorted by key as int,"I have a python dictionary:\nsteps = {""5"":{}, ...","Modifying the sample axis, e.g. removing sampl...",0
2,35763593,2298388,Convert list of lists to list of dictionaries,I want to convert a list of lists to a list of...,Python does not support globals shared between...,0
3,992638,992645,How to setup twill for python 2.6 on Windows?,"I have already downloaded twill 0.9. Also, I h...",easiest way is to just unzip the twill and kee...,1
4,23900344,21077644,How to response a unicode string in flask rest...,I am using flask.ext.rest to build a api. I wa...,"self.set_value_int always returns None, so it'...",0
...,...,...,...,...,...,...
1668355,23409982,23410278,Python regex to get some/not all quote marks o...,i have a .csv file with all fields separated b...,If you don't want to match quotes at beginning...,1
1668356,1750991,37381197,Peeking in a heap in python,What is the official way of peeking in a pytho...,If you want to work with Python objects rather...,0
1668357,12073975,7586821,remove a block of text in a file with python,I am trying to remove a block of text in the a...,"nss3.dll is linked to the following DLLs, whic...",0
1668358,2637038,13403380,how to open dwg file extension with python,I have file with extension .dwg (AutoCAD) and ...,The last element of the energies list is actua...,0
